# [Advent of Code 2024](https://adventofcode.com/2024 )

## General functions

In [ ]:
import re
import itertools
import math
from operator import itemgetter

# import matplotlib.pyplot as plt
# import numpy as np

import requests


In [ ]:
def Input(day, year=2024):
    directory = '{}'.format(year)
    filename = directory + '/input{}.txt'.format(day)
    return open(filename)

def Testdata(day, year=2024):
    directory = '{}'.format(year)
    filename = directory + '/testdata{}.txt'.format(day)
    return open(filename)

def mapt(fn, *args):
    """map(fn, *args) and return the result as a tuple."""
    return tuple(map(fn, *args))

def parse(day, parser=str, sep='\n', output='tuple') -> tuple:
    """Split the day's input file into entries separated by `sep`, and apply `parser` to each."""
    entries = open(f'2024/input{day}.txt').read().rstrip().split(sep)
    return mapt(parser, entries)

In [ ]:
def findroots(a, b, c):
    dis = b * b - 4 * a * c 
    sqrt_val = math.sqrt(abs(dis)) 
    if dis > 0:
        return (-b + sqrt_val)/(2 * a), (-b - sqrt_val)/(2 * a)
    elif dis == 0:
        return -b / (2 * a)
    else:
        return - b / (2 * a), + i, sqrt_val,  - b / (2 * a), - i, sqrt_val

## [Day 1: Historian Hysteria](https://adventofcode.com/2024/day/1)

In [ ]:
def list_diff(lst1, lst2):
    return [abs(e[0] - e[1]) for e in zip(sorted(lst1), sorted(lst2))]

def parselst(str):
    str = list(zip(*[e.split() for e in str.strip().split("\n")]))
    lst1, lst2 = str
    lst1 = [int(e) for e in lst1]
    lst2 = [int(e) for e in lst2]
    return lst1, lst2

In [ ]:
teststr = "3   4\n4   3\n2   5\n1   3\n3   9\n3   3"
assert sum(list_diff(*parselst(teststr)))==11

In [ ]:
inp = Input('01').read()
sum(list_diff(*parselst(inp)))

In [ ]:
def similarity(lst1, lst2):
    total = 0
    for e in lst1:
        total += e * lst2.count(e)
    return total

In [ ]:
assert similarity(*parselst(teststr)) == 31

In [ ]:
similarity(*parselst(inp))